Importamos librerias

In [ ]:
try:
  import google.colab
  !pip install pgmpy
except:
  pass

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

     |████████████████████████████████| 1.9 MB 4.0 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Agregamos los nodos a la red

In [ ]:
model = BayesianNetwork([('Ebola', 'Fiebre'), ('Ebola', 'Sangrado'),('Fiebre', 'Visita'), #BayesianModel is deprecated
                          ('Sangrado','Visita'), ('Sangrado','Complicacion'),('Visita','Doctor')])
print("Nodes: ", model.nodes())
print("Edges: ", model.edges())

Nodes:  ['Ebola', 'Fiebre', 'Sangrado', 'Visita', 'Complicacion', 'Doctor']
Edges:  [('Ebola', 'Fiebre'), ('Ebola', 'Sangrado'), ('Fiebre', 'Visita'), ('Sangrado', 'Visita'), ('Sangrado', 'Complicacion'), ('Visita', 'Doctor')]


Agregamos las probabilidades, para esto dibuje las tablas y la red bayesiana a mano para poder visualizarlo de una manera mas facil, la imagen la anexo en classroom

In [ ]:
dpc_ebola = TabularCPD(variable='Ebola', variable_card=2,
                     values=[[0.01], [0.99]])

dpc_fiebre = TabularCPD(variable='Fiebre', variable_card=2,
                     values=[[0.6, 0.1],
                             [0.4, 0.9]],
                            evidence = ['Ebola'],
                            evidence_card=[2])
                     
dpc_sangrado = TabularCPD(variable='Sangrado', variable_card=2,
                     values=[[0.8, 0.05],
                             [0.2, 0.95]],
                          evidence = ['Ebola'],
                          evidence_card=[2])


dpc_visita = TabularCPD(variable='Visita', variable_card=2,
                     values=[[0.8, 0.5, 0.7, 0],
                             [0.2, 0.5, 0.3, 1]],
                            evidence=['Fiebre', 'Sangrado'],
                            evidence_card=[2, 2])

dpc_complicacion = TabularCPD(variable='Complicacion', variable_card=2,
                     values=[[0.75, 0.1],
                             [0.25, 0.9]],
                              evidence = ['Sangrado'],
                              evidence_card=[2])

dpc_doctor = TabularCPD(variable='Doctor', variable_card=2,
                     values=[[0.6, 0],
                             [0.4, 1]],
                            evidence = ['Visita'],
                            evidence_card=[2])


In [ ]:
model.add_cpds(dpc_ebola, dpc_fiebre, dpc_sangrado, dpc_visita, dpc_doctor,dpc_complicacion)

# verificación del modelo
model.check_model()

True

In [ ]:
model.get_independencies() #podemos checar las independencias

(Sangrado ⟂ Fiebre | Ebola)
(Sangrado ⟂ Doctor | Visita)
(Sangrado ⟂ Fiebre | Ebola, Complicacion)
(Sangrado ⟂ Doctor | Ebola, Visita)
(Sangrado ⟂ Doctor | Visita, Complicacion)
(Sangrado ⟂ Doctor | Visita, Fiebre)
(Sangrado ⟂ Doctor | Ebola, Visita, Complicacion)
(Sangrado ⟂ Doctor | Ebola, Visita, Fiebre)
(Sangrado ⟂ Doctor | Visita, Complicacion, Fiebre)
(Sangrado ⟂ Doctor | Ebola, Visita, Complicacion, Fiebre)
(Ebola ⟂ Complicacion | Sangrado)
(Ebola ⟂ Doctor | Visita)
(Ebola ⟂ Complicacion | Doctor, Sangrado)
(Ebola ⟂ Doctor, Complicacion | Sangrado, Visita)
(Ebola ⟂ Doctor, Visita, Complicacion | Sangrado, Fiebre)
(Ebola ⟂ Doctor | Visita, Complicacion)
(Ebola ⟂ Doctor | Visita, Fiebre)
(Ebola ⟂ Doctor | Sangrado, Visita, Complicacion)
(Ebola ⟂ Doctor, Visita | Sangrado, Complicacion, Fiebre)
(Ebola ⟂ Complicacion | Doctor, Sangrado, Visita)
(Ebola ⟂ Visita, Complicacion | Doctor, Sangrado, Fiebre)
(Ebola ⟂ Doctor, Complicacion | Sangrado, Visita, Fiebre)
(Ebola ⟂ Doctor | Visita

En la funcion query, ponemos nuestras probabilidades, en este caso $$(P(E = Verdadero | D = Verdadero))$$

Supongo que $1=True$ entonces Doctor : 1

In [ ]:
# instanciando objecto de eliminación de variables
var_elimn = VariableElimination(model)

query = var_elimn.query(variables=['Ebola'], evidence={'Doctor': 1})
print('(P(E = Falso | D = Verdadero))\n',query)



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

(P(E = Falso | D = Verdadero))
 +----------+--------------+
| Ebola    |   phi(Ebola) |
+==========+==============+
| Ebola(0) |       0.0063 |
+----------+--------------+
| Ebola(1) |       0.9937 |
+----------+--------------+
